# TP 2

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import matplotlib
import plotly as py
import plotly.express as px 
import matplotlib.ticker as ticker
plt.style.use('default')
import plotly.offline as opy
import plotly.figure_factory as ff
import plotly.io as pio
from wordcloud import WordCloud

sns.set(style="whitegrid")

pd.options.display.float_format = '{:20,.2f}'.format 

import warnings
warnings.filterwarnings('ignore')

In [16]:
#Ejecutar so no funciona el import
#!pip3 install catboost

     |████████████████████████████████| 65.8MB 59kB/s 


In [17]:
from catboost import CatBoostClassifier

## lectura de datos

In [2]:
entrenamiento = pd.read_csv('Training.csv')
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 52 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  16947 non-null  int64  
 1   Region                              16947 non-null  object 
 2   Territory                           16947 non-null  object 
 3   Pricing, Delivery_Terms_Quote_Appr  16947 non-null  int64  
 4   Pricing, Delivery_Terms_Approved    16947 non-null  int64  
 5   Bureaucratic_Code_0_Approval        16947 non-null  int64  
 6   Bureaucratic_Code_0_Approved        16947 non-null  int64  
 7   Submitted_for_Approval              16947 non-null  int64  
 8   Bureaucratic_Code                   16947 non-null  object 
 9   Account_Created_Date                16947 non-null  object 
 10  Source                              16947 non-null  object 
 11  Billing_Country                     16947

In [3]:
testeo = pd.read_csv('Test.csv')
testeo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 51 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  2551 non-null   int64  
 1   Region                              2551 non-null   object 
 2   Territory                           2551 non-null   object 
 3   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   int64  
 4   Pricing, Delivery_Terms_Approved    2551 non-null   int64  
 5   Bureaucratic_Code_0_Approval        2551 non-null   int64  
 6   Bureaucratic_Code_0_Approved        2551 non-null   int64  
 7   Submitted_for_Approval              2551 non-null   int64  
 8   Bureaucratic_Code                   2551 non-null   object 
 9   Account_Created_Date                2551 non-null   object 
 10  Source                              2551 non-null   object 
 11  Billing_Country                     2551 no

## prosesamiento de los df

### modificaciones de oportunidad

In [7]:
def convertirATiempo(df,lista):
    for columna in lista:
        df[columna] = pd.to_datetime(df[columna])
    return df

def convertirATipo(df,lista,tipo):
    for columna in lista:
        df[columna] = df[columna].astype(tipo)
    return df

def convertirAFlotante(df,lista):
    for columna in lista:
        df[columna] = df[columna].apply(lambda x: None if (x == "None" or x == "Other") else float(x))
    return df

def establecerTipos(df):
    """cambia los tipos de las columnas"""
    
    #fechas
    listaDeColumnasFechas = ['Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Last_Modified_Date','Delivery_Year','Planned_Delivery_End_Date','Planned_Delivery_Start_Date']
    df = convertirATiempo(df,listaDeColumnasFechas)
    #enteros
    listaDeColumnasEnteras = ['ID','Opportunity_ID']
    convertirATipo(df,listaDeColumnasEnteras,'int')
    #decimales
    listaDeColumnasEnteras = ['Price','ASP','ASP_(converted)','Total_Taxable_Amount','Total_Amount']
    convertirAFlotante(df,listaDeColumnasEnteras)
    #booleanos
    listaDeColumnasBooleanas = ['Pricing, Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved','Bureaucratic_Code_0_Approval','Bureaucratic_Code_0_Approved']
    convertirATipo(df,listaDeColumnasBooleanas,'bool')
    #categoricas
    listaDeColumnasCategoricas = ['Product_Category_B','Product_Family','Region','Territory','Bureaucratic_Code','Source ','Billing_Country','Account_Name','Opportunity_Name']
    convertirATipo(df,listaDeColumnasCategoricas,'category')
    return df

def eliminarColumnasSinImportancia(df):
    #"Last_Activity" y "Actual_Delivery_Date" son puros nulos
    #"Submitted_for_Approval" puros valores nulos
    #"Prod_Category_A" tiene todos sus valores iguales
    df = df.drop(['Submitted_for_Approval','Last_Activity','Actual_Delivery_Date','Prod_Category_A'], axis=1)
    return df

def agregarColumnas(df):
    #df['Closed'] = df['Stage'].replace({'Closed Won': True, 'Closed Lost': True, 'Negotiation':False,'Proposal':False,'Qualification':False})
    df['Won'] = df['Stage'].replace({'Closed Won': True, 'Closed Lost': False, 'Negotiation':False,'Proposal':False,'Qualification':False})
    convertirATipo(df,['Stage'],'category')
    return df

def correccionDeTerritorios(df):
    #se refiere a diferentes partes de usa
    df = df.replace({'Territory':{'Central US':'USA','NE America':'USA', 'NW America':'USA','SW America':'USA','SE America':'USA'}})
    #
    df = df.replace({'Territory':{'KSA':'Saudi Arabia'}})
    #No se ha encontrado la dorma de los siguientes datos 'Kosovo','South America', 'South East Asia', 'UAE (Dubai)', 'UAE (Others)' (en el caso de kosovo no lo grafica)
    return df

def preparacionDelDataFrame(df,entrenamiento=True):
    df = eliminarColumnasSinImportancia(df)
    df = establecerTipos(df)
    df = correccionDeTerritorios(df)
    df.rename(columns = {'Source ':'Source'}, inplace = True)
    if (entrenamiento):
      df = agregarColumnas(df)
    return df

### por oportunidad

In [5]:
def dataEntrenamiento(df):
  data = df.groupby('Opportunity_ID').agg({#'Won':'sum','Closed':'sum','modificaciones':'sum',
                                           'Won':'sum','modificaciones':'sum',
                                           'Last_Modified_Date':'max','Opportunity_Created_Date':'min',
                                           'Price':'mean','Quote_Expiry_Date':'max',
                                           'Pricing, Delivery_Terms_Quote_Appr':'sum',
                                           'Pricing, Delivery_Terms_Approved':'sum',
                                           'Bureaucratic_Code_0_Approval':'sum',
                                           'Bureaucratic_Code_0_Approved':'sum',
                                           'Account_Created_Date':'min',
                                           'Total_Amount':'sum','Region':'first',
                                           'Territory':'first',
                                           'ASP_(converted)':'sum',
                                           'Product_Type':'first',
                                           'Product_Category_B':'first'})#hay que seguir agregando cosas que son importantes
  return data

def dataTesteo(df):
  data = df.groupby('Opportunity_ID').agg({'modificaciones':'sum',
                                           'Last_Modified_Date':'max','Opportunity_Created_Date':'min',
                                           'Price':'mean','Quote_Expiry_Date':'max',
                                           'Pricing, Delivery_Terms_Quote_Appr':'sum',
                                           'Pricing, Delivery_Terms_Approved':'sum',
                                           'Bureaucratic_Code_0_Approval':'sum',
                                           'Bureaucratic_Code_0_Approved':'sum',
                                           'Account_Created_Date':'min',
                                           'Total_Amount':'sum','Region':'first',
                                           'Territory':'first',
                                           'ASP_(converted)':'sum',
                                           'Product_Type':'first',
                                           'Product_Category_B':'first'})#hay que seguir agregando cosas que son importantes
  return data  

def agrupacionDeOportunidades(df,entrenamiento=True):
  df['modificaciones'] = 1
  if (entrenamiento):
    data = dataEntrenamiento(df)
    #Solo me interesa saber si se cerro con exito
    #data['Closed'] = data['Closed'].agg(lambda x: 1 if x > 0 else 0)
    data['Won'] = data['Won'].agg(lambda x: 1 if x > 0 else 0)
    #data['Lost'] = data['Closed'] - data['Won']#si es won es closed pero si es closed y no es won es perdido
    #data['Lost'] = data['Lost'].agg(lambda x: 1 if x > 0 else 0)
  else:
    data = dataTesteo(df)

  data['Pricing, Delivery_Terms_Quote_Appr'] = data['Pricing, Delivery_Terms_Quote_Appr'].agg(lambda x: 1 if x > 0 else 0)
  data['Pricing, Delivery_Terms_Approved'] = data['Pricing, Delivery_Terms_Approved'].agg(lambda x: 1 if x > 0 else 0)
  data['Bureaucratic_Code_0_Approval'] = data['Bureaucratic_Code_0_Approval'].agg(lambda x: 1 if x > 0 else 0)
  data['Bureaucratic_Code_0_Approved'] = data['Bureaucratic_Code_0_Approved'].agg(lambda x: 1 if x > 0 else 0)
  #data = data.replace({'Won':})
  #data = data[data['Closed'] > 0]#no interesan los que no se cerraron#revisar esta linea despues
  return data

## Ejecucion de cambios

### ejecucion por modificaciones de oportunidad

In [8]:
entrenamiento = preparacionDelDataFrame(entrenamiento)
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 49 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ID                                  16947 non-null  int64         
 1   Region                              16947 non-null  category      
 2   Territory                           16947 non-null  object        
 3   Pricing, Delivery_Terms_Quote_Appr  16947 non-null  bool          
 4   Pricing, Delivery_Terms_Approved    16947 non-null  bool          
 5   Bureaucratic_Code_0_Approval        16947 non-null  bool          
 6   Bureaucratic_Code_0_Approved        16947 non-null  bool          
 7   Bureaucratic_Code                   16947 non-null  category      
 8   Account_Created_Date                16947 non-null  datetime64[ns]
 9   Source                              16947 non-null  category      
 10  Billing_Country       

In [9]:
testeo = preparacionDelDataFrame(testeo,False)
testeo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 47 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ID                                  2551 non-null   int64         
 1   Region                              2551 non-null   category      
 2   Territory                           2551 non-null   object        
 3   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   bool          
 4   Pricing, Delivery_Terms_Approved    2551 non-null   bool          
 5   Bureaucratic_Code_0_Approval        2551 non-null   bool          
 6   Bureaucratic_Code_0_Approved        2551 non-null   bool          
 7   Bureaucratic_Code                   2551 non-null   category      
 8   Account_Created_Date                2551 non-null   datetime64[ns]
 9   Source                              2551 non-null   category      
 10  Billing_Country         

### ejecucion por oportunidad

In [10]:
agrupacionEntrenamiento = agrupacionDeOportunidades(entrenamiento)
agrupacionEntrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9841 entries, 0 to 12803
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Won                                 9841 non-null   int64         
 1   modificaciones                      9841 non-null   int64         
 2   Last_Modified_Date                  9841 non-null   datetime64[ns]
 3   Opportunity_Created_Date            9841 non-null   datetime64[ns]
 4   Price                               267 non-null    float64       
 5   Quote_Expiry_Date                   7489 non-null   datetime64[ns]
 6   Pricing, Delivery_Terms_Quote_Appr  9841 non-null   int64         
 7   Pricing, Delivery_Terms_Approved    9841 non-null   int64         
 8   Bureaucratic_Code_0_Approval        9841 non-null   int64         
 9   Bureaucratic_Code_0_Approved        9841 non-null   int64         
 10  Account_Created_Date   

In [11]:
agrupacionTesteo = agrupacionDeOportunidades(testeo,False)
agrupacionTesteo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567 entries, 10689 to 12368
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   modificaciones                      1567 non-null   int64         
 1   Last_Modified_Date                  1567 non-null   datetime64[ns]
 2   Opportunity_Created_Date            1567 non-null   datetime64[ns]
 3   Price                               110 non-null    float64       
 4   Quote_Expiry_Date                   883 non-null    datetime64[ns]
 5   Pricing, Delivery_Terms_Quote_Appr  1567 non-null   int64         
 6   Pricing, Delivery_Terms_Approved    1567 non-null   int64         
 7   Bureaucratic_Code_0_Approval        1567 non-null   int64         
 8   Bureaucratic_Code_0_Approved        1567 non-null   int64         
 9   Account_Created_Date                1567 non-null   datetime64[ns]
 10  Total_Amount       

## Catboost

In [ ]:
def catboost(train_data,train_labels,cat_features,eval_data)
  # Inicializador de CatBoostClassifier
  model = CatBoostClassifier(iterations=2,
                            learning_rate=1,
                            depth=8)
  #parametros importantes por agregar: l2_leaf_reg, rsm
  #para tipos de category: cat_features, one_hot_max_size

  # Entrenamiento del modelo
  model.fit(train_data, train_labels, cat_features)
  # obtener prediccion
  preds_class = model.predict(eval_data)
  # obtener probabilidad de prediccion
  preds_proba = model.predict_proba(eval_data)
  # obtener RawFormulaVal de la prediccion
  preds_raw = model.predict(eval_data, prediction_type='RawFormulaVal')